In [4]:
import pandas as pd
import numpy as np
import requests
import json
import os
import pprint
# from statesjson import us_states_json
# from states import us_states
from citipy import citipy

In [5]:
DSD = pd.read_csv("Dark_Sky_Data.csv")

In [6]:
DSD

,Country,Limiting Magnitude,Date,Coordinates,Longitude,Latitude
0,United States - Alabama,7,2009-03-22,"[33.0428, -87.394]",-87.3940,33.0428
1,United States - Alabama,7,2009-03-19,"[33.4033, -86.385]",-86.3850,33.4033
2,United States - Alabama,7,2009-03-22,"[32.6994, -86.7865]",-86.7865,32.6994
3,United States - Alabama,7,2009-03-23,"[33.0326, -87.2895]",-87.2895,33.0326
4,United States - Alabama,6,2015-08-13,"[34.7287, -86.7437]",-86.7437,34.7287
...,...,...,...,...,...,...
82492,United States - District of Columbia,-9999,2017-07-21,"[38.9652, -77.0234]",-77.0234,38.9652
82493,United States - District of Columbia,-9999,2010-03-12,"[38.9014, -76.9829]",-76.9829,38.9014
82494,United States - District of Columbia,-9999,2010-03-11,"[38.9012, -77.0007]",-77.0007,38.9012
82495,United States - District of Columbia,-9999,2007-03-22,"[38.95, -77.08]",-77.0800,38.9500


In [7]:
PC = pd.read_csv("Park Coordinates.csv")
pcrounded = PC.round({"Latitude": 4, "Longitude": 3})
#round to make sure data matches between two DFs

In [8]:
pcrounded

,Latitude,Longitude,Park
0,44.3386,-68.273,Acadia National Park
1,-14.2583,-170.683,National Park of American Samoa
2,38.7331,-109.593,Arches National Park
3,43.8554,-102.340,Badlands National Park
4,29.1275,-103.243,Big Bend National Park
...,...,...,...
57,43.6046,-103.421,Wind Cave National Park
58,61.7104,-142.986,Wrangell-St. Elias National Park & Preserve
59,44.4280,-110.588,Yellowstone National Park
60,37.8651,-119.538,Yosemite National Park


In [9]:
def cartesian_product_basic(left, right):
    return (
       DSD.assign(key=1).merge(pcrounded.assign(key=1), on='key').drop('key', 1))
dsd_merged_df = cartesian_product_basic(DSD, pcrounded)
#Credit: Umair Khakoo

In [10]:
dsd_merged_df

,Country,Limiting Magnitude,Date,Coordinates,Longitude_x,Latitude_x,Latitude_y,Longitude_y,Park
0,United States - Alabama,7,2009-03-22,"[33.0428, -87.394]",-87.3940,33.0428,44.3386,-68.273,Acadia National Park
1,United States - Alabama,7,2009-03-22,"[33.0428, -87.394]",-87.3940,33.0428,-14.2583,-170.683,National Park of American Samoa
2,United States - Alabama,7,2009-03-22,"[33.0428, -87.394]",-87.3940,33.0428,38.7331,-109.593,Arches National Park
3,United States - Alabama,7,2009-03-22,"[33.0428, -87.394]",-87.3940,33.0428,43.8554,-102.340,Badlands National Park
4,United States - Alabama,7,2009-03-22,"[33.0428, -87.394]",-87.3940,33.0428,29.1275,-103.243,Big Bend National Park
...,...,...,...,...,...,...,...,...,...
5114809,United States - District of Columbia,-9999,2017-07-20,"[38.9652, -77.0235]",-77.0235,38.9652,43.6046,-103.421,Wind Cave National Park
5114810,United States - District of Columbia,-9999,2017-07-20,"[38.9652, -77.0235]",-77.0235,38.9652,61.7104,-142.986,Wrangell-St. Elias National Park & Preserve
5114811,United States - District of Columbia,-9999,2017-07-20,"[38.9652, -77.0235]",-77.0235,38.9652,44.4280,-110.588,Yellowstone National Park
5114812,United States - District of Columbia,-9999,2017-07-20,"[38.9652, -77.0235]",-77.0235,38.9652,37.8651,-119.538,Yosemite National Park


In [61]:
dsd_df_unclean = dsd_merged_df[
    ((abs(dsd_merged_df["Latitude_x"] - dsd_merged_df["Latitude_y"])) <= 0.44) &\
    ((abs(dsd_merged_df["Longitude_x"] - dsd_merged_df["Longitude_y"])) <= 0.44)
]

In [62]:
dsd_date_unclean = pd.to_datetime(dsd_df["Date"])
dsd_date_df = pd.DataFrame(dsd_date)
dsd_date_df

,Date
25284,2014-03-30
28977,2015-06-13
29039,2015-06-15
29101,2015-06-15
29163,2015-06-15
...,...
5107956,2014-07-25
5108080,2017-08-15
5109010,2015-06-19
5109444,2015-02-01


In [66]:
dsd_date_drop = dsd_df_unclean.drop(columns= "Date")
dsd_df = dsd_date_drop.merge(dsd_date_df, left_index = True, right_index = True)

In [67]:
dsd_df

,Country,Limiting Magnitude,Coordinates,Longitude_x,Latitude_x,Latitude_y,Longitude_y,Park,Date
25284,United States - Alaska,3,"[32.2251, -110.996]",-110.996,32.2251,32.2967,-111.167,Saguaro National Park,2014-03-30
28977,United States - Arizona,7,"[36.0717, -112.2]",-112.200,36.0717,36.1070,-112.113,Grand Canyon National Park,2015-06-13
29039,United States - Arizona,7,"[36.0723, -112.166]",-112.166,36.0723,36.1070,-112.113,Grand Canyon National Park,2015-06-15
29101,United States - Arizona,7,"[36.0721, -112.148]",-112.148,36.0721,36.1070,-112.113,Grand Canyon National Park,2015-06-15
29163,United States - Arizona,7,"[36.0664, -112.146]",-112.146,36.0664,36.1070,-112.113,Grand Canyon National Park,2015-06-15
...,...,...,...,...,...,...,...,...,...
5107956,United States - Wyoming,4,"[43.5873, -110.828]",-110.828,43.5873,43.7410,-110.802,Grand Teton,2014-07-25
5108080,United States - Wyoming,4,"[43.8397, -110.511]",-110.511,43.8397,43.7410,-110.802,Grand Teton,2017-08-15
5109010,United States - Wyoming,2,"[43.9819, -110.663]",-110.663,43.9819,43.7410,-110.802,Grand Teton,2015-06-19
5109444,United States - Wyoming,-999,"[43.4673, -110.787]",-110.787,43.4673,43.7410,-110.802,Grand Teton,2015-02-01


In [68]:
dsd_df_no999 = dsd_df[dsd_df["Limiting Magnitude"] != -999]
dsd_df_clean = dsd_df_no999[dsd_df_no999["Limiting Magnitude"] != -9999]

In [69]:
dsd_df_clean.dtypes

Country                       object
Limiting Magnitude             int64
Coordinates                   object
Longitude_x                  float64
Latitude_x                   float64
Latitude_y                   float64
Longitude_y                  float64
Park                          object
Date                  datetime64[ns]
dtype: object

In [70]:
dsd_df_clean.to_csv("DSDnp.csv", index = False)

In [38]:
dsd_df["Park"].unique()

array(['Saguaro National Park', 'Grand Canyon National Park',
       'Petrified Forest National Park', 'Hot Springs National Park',
       'Pinnacles National Park', 'Joshua Tree National Park',
       'Kings Canyon National Park',
       'Sequoia & Kings Canyon National Parks', 'Yosemite National Park',
       'Death Valley National Park', 'Redwood National and State Parks',
       'Channel Islands National Park', 'Lassen Volcanic National Park',
       'Rocky Mountain National Park',
       'Black Canyon of the Gunnison National Park',
       'Great Sand Dunes National Park and Preserve',
       'Mesa Verde National Park', 'Everglades', 'Biscayne National Park',
       'Dry Tortugas National Park', 'Hawaiʻi Volcanoes National Park',
       'Haleakalā', 'Grand Teton', 'Museum at the Gateway Arch',
       'Indiana Dunes National Park', 'Mammoth Cave',
       'Acadia National Park', 'Voyageurs National Park',
       'Glacier National Park', 'Great Basin National Park',
       'White San